# Predict and create submission file

In [16]:
from ultralytics import YOLO
model = YOLO("/kaggle/input/yolov11l100e/pytorch/default/1/best (6).pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [17]:
LABELS = {i: i for i in range(6)}

In [18]:
import os
import shutil
import json
from glob import glob
import pandas as pd
import cv2

results = model.predict(
    source=test_img_dir,
    imgsz=640,
    conf=0,
    iou=0.6,
    augment=True,
    save=False,
    stream=True,
    batch=16,
    device="0",
    verbose=False,
)

image_predictions_map = {}

all_test_images = sorted(glob(os.path.join(test_img_dir, '*.jpg')))
all_test_image_names = [os.path.basename(p) for p in all_test_images]

for img_name, result in zip(all_test_image_names, results):
    image_stem = img_name.replace('.jpg', '')
    img_path = os.path.join(test_img_dir, img_name)
    img_h, img_w = cv2.imread(img_path).shape[:2]

    boxes = []
    if result.boxes is not None and result.boxes.xyxy is not None:
        box_array = result.boxes.xyxy.cpu().numpy()
        conf_array = result.boxes.conf.cpu().numpy()
        class_array = result.boxes.cls.cpu().numpy()

        for (x_min, y_min, x_max, y_max), conf, class_id in zip(box_array, conf_array, class_array):
            x_min = max(0.0, x_min)
            y_min = max(0.0, y_min)
            x_max = min(float(img_w), x_max)
            y_max = min(float(img_h), y_max)

            if x_max > x_min and y_max > y_min:
                original_id = LABELS.get(int(class_id), int(class_id))
                boxes.append({
                    "x_min": float(x_min),
                    "y_min": float(y_min),
                    "x_max": float(x_max),
                    "y_max": float(y_max),
                    "class": int(original_id),
                    "confidence": float(conf)
                })

    image_predictions_map[image_stem] = boxes


submission_rows = []
for img_stem in all_test_image_names:
    stem = img_stem.replace('.jpg', '')
    submission_rows.append({
        "image_id": stem,
        "bounding_boxes": json.dumps(image_predictions_map.get(stem, []))
    })

submission_df = pd.DataFrame(submission_rows)
submission_path = '/kaggle/working/submission.csv'
submission_df.to_csv(submission_path, index=False)